### unassorted off-target
deeptools?

#### Final unassorted off-target

In [ ]:
df_arts_unassorted

# STOP TODO

homer findMotifsGenome using unassorted (TAS-independent) off-target bed information

In [ ]:
sort_keys = [
    'DetectSeq_SIRT6-DddA11_REP-1', 'DetectSeq_SIRT6-DddA11_REP-2',  # SIRT6
    'DetectSeq_JAK2-DddA11_REP-1', 'DetectSeq_JAK2-DddA11_REP-2',  # JAK2
    'DetectSeq_ATP8-DddA11_REP-1', 'DetectSeq_ATP8-DddA6_REP-1', 'DetectSeq_ATP8-DddAwt_REP-1',  # ATP8
]

# print(df_arts_tale_dep.head(2))
# print('-' * 100)
# print(df_arts.head(2))
df_arts

##### 

In [ ]:
ls = []
for sample in sort_keys:
    tas_dep_index = sorted(df_arts_tale_dep.query('`<sample>`==@sample')['region_index'].unique().tolist())
    tas_indep_index = sorted(df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')[
                                 'region_index'].unique().tolist())
    ls.append([sample, len(tas_dep_index), len(tas_indep_index), len(tas_dep_index) + len(tas_indep_index)])
    # |    | <sample>                     |   TAS-dep |   putative TAS-indep |   Total |
    # |---:|:-----------------------------|----------:|---------------------:|--------:|
    # |  0 | DetectSeq_SIRT6-DddA11_REP-1 |        68 |                 3331 |    3399 |
    # |  1 | DetectSeq_SIRT6-DddA11_REP-2 |       230 |                 9163 |    9393 |
    # |  2 | DetectSeq_JAK2-DddA11_REP-1  |        24 |                  797 |     821 |
    # |  3 | DetectSeq_JAK2-DddA11_REP-2  |       184 |                  573 |     757 |
    # |  4 | DetectSeq_ATP8-DddA11_REP-1  |       128 |                   87 |     215 |
    # |  5 | DetectSeq_ATP8-DddA6_REP-1   |       512 |                  260 |     772 |
    # |  6 | DetectSeq_ATP8-DddAwt_REP-1  |        18 |                   22 |      40 |

    df_out = df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')

    df_out[['chrom', 'start', 'end']] = df_out['region_index'].str.split('_', expand=True)
    df_out = df_out[['chrom', 'start', 'end', 'region_index', 'align_strand']].copy()
    df_out.insert(4, 'score', '.')
    df_out['start'] = df_out['start'].astype(int) - 30
    df_out['end'] = df_out['end'].astype(int) + 30
    df_out.to_csv(f'../art/from_art_putative_IND_{sample}.bed', header=False, index=False, sep='\t')
print(pd.DataFrame(ls, columns=['<sample>', 'TAS-dep', 'putative TAS-indep', 'Total']).to_markdown())

```bash
findMotifsGenome.pl from_art_putative_IND_DetectSeq_ATP6.1-DddSs.bed $HOME/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa homer_putative_IND_DetectSeq_ATP6.1-DddSs -size 50 -mask
...
...
...
```

## homer annotatePeaks for all off-target

```bash
cd final_list_after_igv_check

annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA6_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddAwt_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-2.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-2.homer_log &

bioat table merge -input_fmt tsv -output_fmt csv --input_header True --output_header True ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log homer.annoResults.merge.csv
```

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../annoResults/homer.annoResults.merge.csv')

filt = ['Promoter', '5UTR', '3UTR', 'Exon', 'Intron', 'TTS', 'Intergenic']

df = df[df['Total size (bp)'].str.isdecimal()].copy()
df.drop_duplicates(subset=['<sample>', 'Annotation'], keep='first', inplace=True)
df = df[df['Annotation'].map(lambda x: x in filt)].copy()
df.iloc[:, 2:] = df.iloc[:, 2:].astype(float)
df

In [ ]:
from lets_plot import *
from lets_plot.mapping import as_discrete

In [ ]:
LetsPlot.setup_html()

In [ ]:
(
        ggplot(data=df, mapping=aes(x=as_discrete("Annotation", order=-1), y="Log2 Ratio (obs/exp)", fill='<sample>'))
        + geom_bar(stat='identity', position='dodge', width=0.7)
        # + scale_x_discrete(labels=filt)
        + ggsize(1000, 500)
    # + facet_grid(y='<sample>', scales='free')
    # + scale_y_continuous(limits=(-2, 10))
)

### unassorted off-target
deeptools?

#### Final unassorted off-target

In [ ]:
df_arts_unassorted

# STOP TODO

homer findMotifsGenome using unassorted (TAS-independent) off-target bed information

In [ ]:
sort_keys = [
    'DetectSeq_SIRT6-DddA11_REP-1', 'DetectSeq_SIRT6-DddA11_REP-2',  # SIRT6
    'DetectSeq_JAK2-DddA11_REP-1', 'DetectSeq_JAK2-DddA11_REP-2',  # JAK2
    'DetectSeq_ATP8-DddA11_REP-1', 'DetectSeq_ATP8-DddA6_REP-1', 'DetectSeq_ATP8-DddAwt_REP-1',  # ATP8
]

# print(df_arts_tale_dep.head(2))
# print('-' * 100)
# print(df_arts.head(2))
df_arts

##### 

In [ ]:
ls = []
for sample in sort_keys:
    tas_dep_index = sorted(df_arts_tale_dep.query('`<sample>`==@sample')['region_index'].unique().tolist())
    tas_indep_index = sorted(df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')[
                                 'region_index'].unique().tolist())
    ls.append([sample, len(tas_dep_index), len(tas_indep_index), len(tas_dep_index) + len(tas_indep_index)])
    # |    | <sample>                     |   TAS-dep |   putative TAS-indep |   Total |
    # |---:|:-----------------------------|----------:|---------------------:|--------:|
    # |  0 | DetectSeq_SIRT6-DddA11_REP-1 |        68 |                 3331 |    3399 |
    # |  1 | DetectSeq_SIRT6-DddA11_REP-2 |       230 |                 9163 |    9393 |
    # |  2 | DetectSeq_JAK2-DddA11_REP-1  |        24 |                  797 |     821 |
    # |  3 | DetectSeq_JAK2-DddA11_REP-2  |       184 |                  573 |     757 |
    # |  4 | DetectSeq_ATP8-DddA11_REP-1  |       128 |                   87 |     215 |
    # |  5 | DetectSeq_ATP8-DddA6_REP-1   |       512 |                  260 |     772 |
    # |  6 | DetectSeq_ATP8-DddAwt_REP-1  |        18 |                   22 |      40 |

    df_out = df_arts.query('`<sample>`==@sample').query('region_index not in @tas_dep_index')

    df_out[['chrom', 'start', 'end']] = df_out['region_index'].str.split('_', expand=True)
    df_out = df_out[['chrom', 'start', 'end', 'region_index', 'align_strand']].copy()
    df_out.insert(4, 'score', '.')
    df_out['start'] = df_out['start'].astype(int) - 30
    df_out['end'] = df_out['end'].astype(int) + 30
    df_out.to_csv(f'../art/from_art_putative_IND_{sample}.bed', header=False, index=False, sep='\t')
print(pd.DataFrame(ls, columns=['<sample>', 'TAS-dep', 'putative TAS-indep', 'Total']).to_markdown())

```bash
findMotifsGenome.pl from_art_putative_IND_DetectSeq_ATP6.1-DddSs.bed $HOME/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa homer_putative_IND_DetectSeq_ATP6.1-DddSs -size 50 -mask
...
...
...
```

## homer annotatePeaks for all off-target

```bash
cd final_list_after_igv_check

annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddA6_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddA6_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_ATP8-DddAwt_REP-1_final_list.mpmat hg38 -annStats ../annoResults/ATP8-DddAwt_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_JAK2-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/JAK2-DddA11_REP-2.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-1_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-1.homer_log &
annotatePeaks.pl 2023-04-20_DetectSeq_SIRT6-DddA11_REP-2_final_list.mpmat hg38 -annStats ../annoResults/SIRT6-DddA11_REP-2.homer_log &

bioat table merge -input_fmt tsv -output_fmt csv --input_header True --output_header True ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log ATP8-DddA11_REP-1.homer_log,ATP8-DddA6_REP-1.homer_log,ATP8-DddAwt_REP-1.homer_log,JAK2-DddA11_REP-1.homer_log,JAK2-DddA11_REP-2.homer_log,SIRT6-DddA11_REP-1.homer_log,SIRT6-DddA11_REP-2.homer_log homer.annoResults.merge.csv
```

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../annoResults/homer.annoResults.merge.csv')

filt = ['Promoter', '5UTR', '3UTR', 'Exon', 'Intron', 'TTS', 'Intergenic']

df = df[df['Total size (bp)'].str.isdecimal()].copy()
df.drop_duplicates(subset=['<sample>', 'Annotation'], keep='first', inplace=True)
df = df[df['Annotation'].map(lambda x: x in filt)].copy()
df.iloc[:, 2:] = df.iloc[:, 2:].astype(float)
df

In [ ]:
from lets_plot import *
from lets_plot.mapping import as_discrete

In [ ]:
LetsPlot.setup_html()

In [ ]:
(
        ggplot(data=df, mapping=aes(x=as_discrete("Annotation", order=-1), y="Log2 Ratio (obs/exp)", fill='<sample>'))
        + geom_bar(stat='identity', position='dodge', width=0.7)
        # + scale_x_discrete(labels=filt)
        + ggsize(1000, 500)
    # + facet_grid(y='<sample>', scales='free')
    # + scale_y_continuous(limits=(-2, 10))
)